# Crafting robust LLM-as-a-judge scorers

Let's say you're working on a customer service bot and trying to evaluate the quality of its outputs. Consider a question like "What is your return policy?". If the ground truth answer is "You can return
items within 30 days of purchase", and your bot generates the answer "You can return items within 30 days", a direct comparison of the two strings would say they're not equal,
but an LLM-as-a-judge scorer can understand that they are in fact the same.

## Installing dependencies

Let's install a few basic dependencies. We'll use the CoQA dataset (via DuckDB), Braintrust for evals, and OpenAI's LLMs.


In [2]:
%pip install autoevals duckdb braintrust openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 42.2 MB/s eta 0:00:001m26.8 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


## Explore the dataset


In [11]:
import duckdb

con = duckdb.connect(":memory:")
result = con.query("""
    SELECT * FROM 'hf://datasets/stanfordnlp/coqa/data/validation-00000-of-00001.parquet'
        LIMIT 1
""").fetchall()[0]

print("Passage:")
print(result[1])

print("\nQuestion:")
print(result[2][0])

print("\nAnswer:")
print(result[3]["input_text"][0])

Passage:
Once upon a time, in a barn near a farm house, there lived a little white kitten named Cotton. Cotton lived high up in a nice warm place above the barn where all of the farmer's horses slept. But Cotton wasn't alone in her little home above the barn, oh no. She shared her hay bed with her mommy and 5 other sisters. All of her sisters were cute and fluffy, like Cotton. But she was the only white one in the bunch. The rest of her sisters were all orange with beautiful white tiger stripes like Cotton's mommy. Being different made Cotton quite sad. She often wished she looked like the rest of her family. So one day, when Cotton found a can of the old farmer's orange paint, she used it to paint herself like them. When her mommy and sisters found her they started laughing. 

"What are you doing, Cotton?!" 

"I only wanted to be more like you". 

Cotton's mommy rubbed her face on Cotton's and said "Oh Cotton, but your fur is so pretty and special, like you. We would never want you to

In [ ]:
import os

import braintrust
import duckdb
from openai import OpenAI

client = braintrust.wrap_openai(
    OpenAI(
        api_key=os.environ["OPENAI_API_KEY"],
        base_url="https://api.braintrust.dev/v1/proxy",  # Caches requests to OpenAI
    )
)